In [3]:
from langchain.prompts import ChatPromptTemplate
SYSTEM_PROMPT = '''You are an expert sommelier with extensive knowledge in wine, wine pairing, and
    the intricacies of food and beverage service. Your primary role is to assist users
    in selecting the best wines and pairing them perfectly with meals. You have a deep
    understanding of various wine regions, grape varieties, wine production methods, and
    current trends in the industry. You possess a refined palate, able to discern subtle
    flavors and characteristics in wines. Your advice is always clear, approachable, and
    tailored to each user’s preferences and specific dining context. You also educate users
    on wine appreciation, proper wine service, and the art of creating a harmonious dining
    experience. Your demeanor is professional, courteous, and passionate about wine culture,
    aiming to make each wine selection and pairing a memorable experience for users.'''

wine_query = '이 와인에 어울리는 요리에는 어떤 것들이 있을까요?'

chat_template = ChatPromptTemplate.from_messages(
    [
    ('system',SYSTEM_PROMPT),
    ('human',[{'type': 'text', 'text':'{text}'},
              {'type':'image_url', 'image_url': {'url': '{image_url}'}}])# 사용자 프롬프트 
    ]
)

chat_template


ChatPromptTemplate(input_variables=['image_url', 'text'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are an expert sommelier with extensive knowledge in wine, wine pairing, and\n    the intricacies of food and beverage service. Your primary role is to assist users\n    in selecting the best wines and pairing them perfectly with meals. You have a deep\n    understanding of various wine regions, grape varieties, wine production methods, and\n    current trends in the industry. You possess a refined palate, able to discern subtle\n    flavors and characteristics in wines. Your advice is always clear, approachable, and\n    tailored to each user’s preferences and specific dining context. You also educate users\n    on wine appreciation, proper wine service, and the art of creating a harmonious dining\n    experience. Your demeanor is professional, courteous, and pa

In [4]:
# llm 정의 
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model='gpt-4o-mini')

In [5]:
# outputparser 정의
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()

In [6]:
chain = chat_template|llm|StrOutputParser()
response = chain.invoke(
    {
        'text': wine_query,
        'image_url': 'https://images.vivino.com/thumbs/Z90I3--JRKWlpMA8wdLY-Q_pb_x600.png'
    }
)

response

'Masserì Primitivo는 이탈리아의 풀 바디 레드 와인으로, 풍부한 과일 맛과 부드러운 탄닌이 특징입니다. 이 와인에 어울리는 요리로는 다음과 같은 것들이 있습니다:\n\n1. **빅안 구이**: 소고기, 양고기, 혹은 돼지고기를 적절히 조리하여 그 맛을 극대화합니다.\n2. **파스타**: 토마토 소스, 미트 소스가 곁들여진 파스타와 잘 매칭됩니다.\n3. **치즈**: 구울이나 고르곤졸라와 같은 강한 맛의 치즈가 이 와인과 조화롭게 어울립니다.\n4. **바비큐 요리**: 연기가 있는 바비큐나 그릴에 구운 고기와의 조합이 훌륭합니다.\n5. **스파이시 요리**: 스파이시한 이탈리안 소시지나 매운 요리와도 잘 어울립니다.\n\n이 와인은 깊고 복합적인 맛을 지니고 있어 다양한 요리와의 조화가 좋습니다. 적절한 식사와 함께 즐기시면 더욱 좋습니다!'

In [7]:
def recommend_dishes_chain(query):
    chat_template = ChatPromptTemplate.from_messages(
    [
    ('system',SYSTEM_PROMPT),
    ('human',[{'type': 'text', 'text':query['text']}, # 입력 받은 쿼리에서 text를 꺼내
              {'type':'image_url', 'image_url': {'url': query['image_url']}}])# 사용자 프롬프트 
    ]
    )   
    wine_chain = chat_template|llm|StrOutputParser()
    return wine_chain

In [8]:
# 함수 호출 
query_1 = {
    'text':wine_query,
    'image_url':'https://images.vivino.com/thumbs/Z90I3--JRKWlpMA8wdLY-Q_pb_x600.png'
}
rec_dish_chain = recommend_dishes_chain(query_1)

In [9]:
rec_dish_chain.invoke(query_1)

'Masserì Primitivo는 풍부하고 진한 맛을 가진 레드 와인입니다. 이 와인과 잘 어울리는 요리로는 다음과 같은 것들이 있습니다:\n\n1. **고기 요리**: 소고기 스테이크, 양고기, 혹은 바베큐와 같은 진한 육류 요리와 잘 어울립니다.\n2. **파스타**: 진한 토마토 소스나 고기 소스를 곁들인 파스타, 특히 볼로네제 스타일의 요리와 좋은 조화를 이룹니다.\n3. **리조또**: 고기나 버섯을 사용한 리조또는 와인의 풍미와 잘 어울립니다.\n4. **치즈**: 숙성된 파르메산, 체다, 혹은 고르곤졸라와 같은 강한 맛의 치즈와 매칭하면 좋습니다.\n5. **스튜**: 이탈리안 스타일의 고기 스튜는 이 와인의 풍미를 더욱 강조해줍니다.\n\n이 와인은 일반적으로 과일의 풍미가 강하고 약간의 스파이스 노트를 가지고 있어, 이러한 요리들과의 조화가 매우 뛰어납니다. 감사한 식사 되세요!'

In [10]:
from langchain_core.runnables import RunnableLambda
runnable = RunnableLambda(recommend_dishes_chain)
response = runnable.invoke(query_1)
response

'Masserì Primitivo는 일반적으로 진하고 풍부한 과일 맛과 부드러운 타닌을 가진 레드 와인입니다. 이 와인과 잘 어울리는 요리로는 다음과 같은 것들이 있습니다:\n\n1. **구운 고기**: 특히 소고기 스테이크나 양갈비와 잘 어울립니다. 그릴에서 구워낸 고기의 풍미가 와인의 과일과 잘 조화를 이룹니다.\n\n2. **이탈리안 파스타**: 토마토 소스 기반의 파스타나, 고기소스 같은 리치한 소스가 있는 요리와 특히 좋습니다.\n\n3. **치즈**: 숙성된 치즈, 예를 들어 파르미지아노 레지아노나 고르곤졸라 같은 블루치즈와 잘 맞습니다.\n\n4. **장아찌와 나치 등**: 올리브나 마르inated 채소와 같은 감칠맛 있는 안주와도 좋은 조화를 이룹니다.\n\n5. **스파이시 요리**: 매운 스튜나 바비큐 소스를 곁들인 요리 역시 와인의 구조와 좋은 균형을 이룹니다.\n\n이 와인의 풍미에 맞춰 다양한 요리를 시도해보시면 멋진 매칭을 경험하실 수 있습니다.'

In [11]:
# describe_dish_flavor_chain 용 랭체인 프롬프트 

In [12]:
dish_system_prompt = """
            Persona:
            As a flavor analysis system, I am equipped with a deep understanding of food ingredients, cooking methods, and sensory properties such as taste, texture, and aroma. I can assess and break down the flavor profiles of dishes by identifying the dominant tastes (sweet, sour, salty, bitter, umami) as well as subtler elements like spice levels, richness, freshness, and aftertaste. I am able to compare different foods based on their ingredients and cooking techniques, while also considering cultural influences and typical pairings. My goal is to provide a detailed analysis of a dish’s flavor profile to help users better understand what makes it unique or to aid in choosing complementary foods and drinks.

            Role:

            1. Flavor Identification: I analyze the dominant and secondary flavors of a dish, highlighting key taste elements such as sweetness, acidity, bitterness, saltiness, umami, and the presence of spices or herbs.
            2. Texture and Aroma Analysis: Beyond taste, I assess the mouthfeel and aroma of the dish, taking into account how texture (e.g., creamy, crunchy) and scents (e.g., smoky, floral) contribute to the overall experience.
            3. Ingredient Breakdown: I evaluate the role each ingredient plays in the dish’s flavor, including their impact on the dish's balance, richness, or intensity.
            4. Culinary Influence: I consider the cultural or regional influences that shape the dish, understanding how traditional cooking methods or unique ingredients affect the overall taste.
            5. Food and Drink Pairing: Based on the dish's flavor profile, I suggest complementary food or drink pairings that enhance or balance the dish’s qualities.

            Examples:

            - Dish Flavor Breakdown:
            For a butter garlic shrimp, I identify the richness from the butter, the pungent aroma of garlic, and the subtle sweetness of the shrimp. The dish balances richness with a touch of saltiness, and the soft, tender texture of the shrimp is complemented by the slight crispness from grilling.

            - Texture and Aroma Analysis:
            A creamy mushroom risotto has a smooth, velvety texture due to the creamy broth and butter. The earthy aroma from the mushrooms enhances the umami flavor, while a sprinkle of Parmesan adds a savory touch with a mild sharpness.

            - Ingredient Role Assessment:
            In a spicy Thai curry, the coconut milk provides a rich, creamy base, while the lemongrass and lime add freshness and citrus notes. The chilies bring the heat, and the balance between sweet, sour, and spicy elements creates a dynamic flavor profile.

            - Cultural Influence:
            A traditional Italian margherita pizza draws on the classic combination of fresh tomatoes, mozzarella, and basil. The simplicity of the ingredients allows the flavors to shine, with the tanginess of the tomato sauce balancing the richness of the cheese and the freshness of the basil.

            - Food Pairing Example:
            For a rich chocolate cake, I would recommend a sweet dessert wine like Port to complement the bitterness of the chocolate, or a light espresso to contrast the sweetness and enhance the richness of the dessert.
        """

In [13]:
def describe_dish_flavor_chain(query):
    # 이미지가 있는 경우와 없는 경우를 구분하여 처리
    if query.get("image_urls"):
        # 이미지가 있는 경우: 멀티모달 프롬프트
        messages = [
            ("system", dish_system_prompt),
            ("human", [ #"human" 으로 해도 무방 - 역할 태그의 차이가 크지 않음
                {"type": "text", "text": "이 요리의 이름과 맛을 한 문장으로 요약해주세요."},
                *[{"type": "image_url", "image_url": {"url": url}} for url in query["image_urls"]]
            ])
        ]
    else:
        # 텍스트만 있는 경우 (이미지 없이는 요리 설명이 어려우므로 안내 메시지)
        messages = [
            ("system", dish_system_prompt),
            ("user", "요리 이미지가 제공되지 않았습니다. 요리 이미지를 업로드해주시면 정확한 분석을 도와드리겠습니다.")
        ]
    
    prompt = ChatPromptTemplate.from_messages(messages)
    output_parser = StrOutputParser()
    
    chain = prompt | llm | output_parser
    return chain

In [14]:
runnable = RunnableLambda(describe_dish_flavor_chain)
response = runnable.invoke({
    "image_urls": ["https://www.stockfood.com/Sites/StockFood/Documents/Homepage/News//en/16.jpg"]
})

print(response)

이 요리는 피스타치오 오트 바로, 고소한 피스타치오와 달콤한 오트밀의 조화로 바삭하고 씹는 맛이 느껴지는 간식입니다.


In [ ]:
# 질의 임베딩 
from dotenv import load_dotenv
import os

load_dotenv()

OPENAI_API_KEY = os.environ['OPENAI_API_KEY']
PINECONE_API_KEY = os.environ['PINECONE_API_KEY']


c:\Users\Admin\miniconda3\envs\prompting_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


0 page_content=': 0
country: Italy
description: Aromas include tropical fruit, broom, brimstone and dried herb. The palate isn't overly expressive, offering unripened apple, citrus and dried sage alongside brisk acidity.
designation: Vulkà Bianco
points: 87
price: 
province: Sicily & Sardinia
region_1: Etna
region_2: 
taster_name: Kerin O’Keefe
taster_twitter_handle: @kerinokeefe
title: Nicosia 2013 Vulkà Bianco  (Etna)
variety: White Blend
winery: Nicosia' metadata={'source': 'winemag-data-130k-v2.csv', 'row': 0}
1 page_content=': 1
country: Portugal
description: This is ripe and fruity, a wine that is smooth while still structured. Firm tannins are filled out with juicy red berry fruits and freshened with acidity. It's  already drinkable, although it will certainly be better from 2016.
designation: Avidagos
points: 87
price: 15.0
province: Douro
region_1: 
region_2: 
taster_name: Roger Voss
taster_twitter_handle: @vossroger
title: Quinta dos Avidagos 2011 Avidagos Red (Douro)
variety

c:\Users\Admin\miniconda3\envs\prompting_env\Lib\site-packages\langchain_pinecone\__init__.py:3: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_pinecone.vectorstores import Pinecone, PineconeVectorStore


{'dimension': 1536,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'wine': {'vector_count': 130099}},
 'total_vector_count': 130099,
 'vector_type': 'dense'}

In [ ]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=PINECONE_API_KEY)
# 'wiki' 인덱스를 가져옵니다.
index_name = "wine-review"
index = pc.Index(index_name)


In [ ]:
index.describe_index_stats()

In [ ]:
from langchain_community.document_loaders import CSVLoader

loader = CSVLoader("winemag-data-130k-v2.csv")
docs = loader.load()

for i, doc in enumerate(docs[:3]):
    print(str(i), doc)


In [ ]:
from langchain_openai import OpenAIEmbeddings
import os
embedding = OpenAIEmbeddings(model="text-embedding-3-small")


In [ ]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(
    index_name="wine-review",   # 기존에 만들어둔 인덱스 이름
    embedding=embedding,
    namespace="wine"            # 기존에 저장해둔 namespace
)


In [ ]:
results = vector_store.similarity_search(
    "달콤한 맛을 느낄 수 있는 와인", 
    k=5, 
    namespace="wine"
)

results


In [ ]:
index.describe_index_stats()

In [ ]:
def search_wine(dish_flavor):
    results = vector_store.similarity_search(
        dish_flavor, 
        k=5, 
        namespace='wine'
    )

    return {
        "dish_flavor": dish_flavor,
        "wine_reviews": "\n".join([doc.page_content for doc in results])
    }



이 요리는 판차넬라 샐러드로, 신선한 토마토와 바질의 상큼함이 빵의 고소함과 어우러져 상쾌하고 풍부한 맛을 냅니다.



In [ ]:
# vector store에서 현재 요리의 풍미와 맛과 유사한 와인 검색
taste_query =  "이 요리는 판차넬라 샐러드로, 신선한 토마토와 바질의 상큼함이 빵의 고소함과 어우러져 상쾌하고 풍부한 맛을 냅니다."
results = vector_store.similarity_search(
  taste_query, 
    k=5, 
    namespace='wine'
)

results


In [ ]:
from langchain_core.runnables import RunnableLambda

runnable = RunnableLambda(search_wine)
response = runnable.invoke(taste_query)
print(response['dish_flavor'])
print(response['wine_reviews'])



In [ ]:
WINE_SOMMELIER_SYSTEM = """
            Persona:

            As a sommelier, I possess an extensive knowledge of wines, including grape varieties, regions, tasting notes, and food pairings. I am highly skilled in recommending wines based on individual preferences, specific occasions, and particular dishes. My expertise includes understanding wine production methods, flavor profiles, and how they interact with different foods. I also stay updated on the latest trends in the wine world and am capable of suggesting wines that are both traditional and adventurous. I strive to provide personalized, thoughtful recommendations to enhance the dining experience.

            Role:

            1. Wine & Food Pairing: I offer detailed wine recommendations that pair harmoniously with specific dishes, balancing flavors and enhancing the overall dining experience. Whether it's a simple snack or an elaborate meal, I suggest wines that complement the texture, taste, and style of the food.
            2. Wine Selection Guidance: For various occasions (celebrations, formal dinners, casual gatherings), I assist in selecting wines that suit the event and align with the preferences of the individuals involved.
            3. Wine Tasting Expertise: I can help identify wines based on tasting notes like acidity, tannin levels, sweetness, and body, providing insights into what makes a wine unique.
            4. Explaining Wine Terminology: I simplify complex wine terminology, making it easy for everyone to understand grape varieties, regions, and tasting profiles.
            5. Educational Role: I inform and educate about different wine regions, production techniques, and wine styles, fostering an appreciation for the diversity of wines available.

            Examples:

            - Wine Pairing Example (Dish First):
            For a grilled butter garlic shrimp dish, I would recommend a Sauvignon Blanc or a Chardonnay with crisp acidity to cut through the richness of the butter and enhance the seafood’s flavors.

            - Wine Pairing Example (Wine First):  
            If you're enjoying a Cabernet Sauvignon, its bold tannins and dark fruit flavors pair wonderfully with grilled steak or lamb. The richness of the meat complements the intensity of the wine.

            - Wine Pairing Example (Wine First):
            A Pinot Noir, known for its lighter body and subtle flavors of red berries, is perfect alongside roasted duck or mushroom risotto, as its earthy notes complement the dishes.

            - Occasion-Based Selection:
            If you are celebrating a romantic anniversary dinner, I would suggest a classic Champagne or an elegant Pinot Noir, perfect for a special and intimate evening.

            - Guiding by Taste Preferences:
            If you enjoy wines with bold flavors and intense tannins, a Cabernet Sauvignon from Napa Valley would suit your palate perfectly. For something lighter and fruitier, a Riesling could be a delightful alternative, pairing well with spicy dishes or fresh salads.
        """


In [ ]:

def recommend_wine_chain(query):
    prompt = ChatPromptTemplate.from_messages([
        ("system", WINE_SOMMELIER_SYSTEM),
        ("human", """
            와인 페이링 추천에 아래 요리/맛, 와인 리뷰만을 참고하여 한글로 답변해 주시기 바랍니다.

            요리/맛:
            {dish_flavor}

            와인 리뷰:
            {wine_reviews}
        """)
    ])

    output_parser = StrOutputParser()
    chain = prompt | llm | output_parser
    
    return chain

In [25]:
# chain 연결 
runnable_1 = RunnableLambda(describe_dish_flavor_chain)
runnable_2 = RunnableLambda(search_wine)
runnable_3 = RunnableLambda(recommend_wine_chain)

chain = runnable_1 | runnable_2 | runnable_3

In [26]:
# 음식이미지 링크 > 음식찾아서 음식맛묘사 
# > 음식맛이 와인평에 있는 와인 검색  
# > 와인평+음식맛 > 소믈리에메시지
chain.get_graph().print_ascii()


+----------------------------------+ 
| describe_dish_flavor_chain_input | 
+----------------------------------+ 
                  *                  
                  *                  
                  *                  
           +------------+            
           | ChatOpenAI |            
           +------------+            
                  *                  
                  *                  
                  *                  
          +-------------+            
          | search_wine |            
          +-------------+            
                  *                  
                  *                  
                  *                  
           +------------+            
           | ChatOpenAI |            
           +------------+            
                  *                  
                  *                  
                  *                  
  +-----------------------------+    
  | recommend_wine_chain_output |    
  +---------

In [27]:
response = chain.invoke(
  {
    "image_url": "https://www.stockfood.com/Sites/StockFood/Documents/Homepage/News//en/16.jpg"
  }
)
response

'요리와 관련된 정보가 부족한 상태입니다. 추천하고 싶은 특정 요리의 이름이나 주요 재료를 말씀해 주시면, 이에 맞춰 와인을 추천해 드리겠습니다. 어떤 요리를 고려하고 계신가요?'

In [ ]:
# 국가별 필터링
def search_wine_by_country(query, country="US"):
    results = vector_store.similarity_search(
        query,
        k=5,
        filter={"country": country},
        namespace="wine"
    )
    return results

In [32]:
query = "달콤한 와인 찾아줘"
search_wine_by_country(query)

[]